In [1]:
import openpathsampling as paths
storage = paths.storage.Storage("mistis.nc", "r")

In [2]:
mistis = storage.networks.load(0)

In [3]:
for t in mistis.transitions:
    print t

RETISTransition: A->B
[IntersectionVolume] -> [IntersectionVolume] or [IntersectionVolume]
Interface: -inf<opX<-0.3
Interface: -inf<opX<-0.2
Interface: -inf<opX<-0.1
Interface: -inf<opX<0.0

RETISTransition: A->C
[IntersectionVolume] -> [IntersectionVolume] or [IntersectionVolume]
Interface: -inf<opY<-0.3
Interface: -inf<opY<-0.2
Interface: -inf<opY<-0.1
Interface: -inf<opY<0.0

RETISTransition: B->A
[IntersectionVolume] -> [IntersectionVolume] or [IntersectionVolume]
Interface: -inf<opXprime<-0.3
Interface: -inf<opXprime<-0.2
Interface: -inf<opXprime<-0.1
Interface: -inf<opXprime<0.0



In [4]:
print mistis.movers

{'repex': [<openpathsampling.pathmover.ReplicaExchangeMover object at 0x11a778e90>, <openpathsampling.pathmover.ReplicaExchangeMover object at 0x11a7c8990>, <openpathsampling.pathmover.ReplicaExchangeMover object at 0x11a778f50>, <openpathsampling.pathmover.ReplicaExchangeMover object at 0x11a7c85d0>, <openpathsampling.pathmover.ReplicaExchangeMover object at 0x11a778990>, <openpathsampling.pathmover.ReplicaExchangeMover object at 0x11a7c82d0>, <openpathsampling.pathmover.ReplicaExchangeMover object at 0x11a778150>], 'msouter_shooting': [<openpathsampling.pathmover.OneWayShootingMover object at 0x11a7ba490>], 'msouter_pathreversal': [<openpathsampling.pathmover.PathReversalMover object at 0x11a785490>], 'msouter_repex': [<openpathsampling.pathmover.ReplicaExchangeMover object at 0x11a760350>, <openpathsampling.pathmover.ReplicaExchangeMover object at 0x11a785b50>], 'shooting': [<openpathsampling.pathmover.OneWayShootingMover object at 0x11a785ed0>, <openpathsampling.pathmover.OneWaySho